# The reduced decision making model by Wong-Wang
The equations and parameters are described in the body, appendix and supplementary materials of the following paper:

<i>Wong, Kong-Fatt and Xiao-Jing Wang.</i> <b>"A recurrent network mechanism  of time integration in perceptual decisions"</b>. Journal of Neuroscience 26, no. 4 (2006)

In [3]:
import numpy as np
import random

#Single-cell input-output relation [Appendix & Supplementary material D]
def input_output_relation(state,a=270,b=108,d=0.154):
    return (a*state - b)/(1-np.exp(-d*(a*state - b)))
#H(x) = (a * x - b) / (1 - exp(-d*(a * x - b)))

#Noisy sources from each ionic channel 
def noisy_channel(noises,tau_AMPA = 2, s_noise=0.05):
    #a two-dimensional array of noisy displacements for the current run
    eta =  np.random.rand(2) * np.sqrt(tau_AMPA * (s_noise ** 2))
    #Inoise1' = -Inoise1 + eta1 * sqrt(tau_AMPA * s_noise^2)
    #Inoise2' = -Inoise2 + eta2 * sqrt(tau_AMPA * s_noise^2)
    return -noises + eta #[ionic_noises]

#Summing contributions to the hidden state (x) of each population
#x1 = JN11 * S1 - JN12 * S2 + I0 + I1 + Inoise1
#x2 = JN22 * S2 - JN21 * S1 + I0 + I2 + Inoise2

def build_hidden_states(couplings,current_states,inputs,ionic_noises,external_sources = I0):
    #returns a 2x1 array, each element being the hidden state activity (firing rate) of one excitatory population
    return couplings @ current_states + inputs + ionic_noises + external_sources

#yet to port/solve

#these two will become the function below
#S1' = -S1/tau_S + (1 - S1) * g * H(x1)/1000
#S2' = -S2/tau_S + (1 - S2) * g * H(x2)/1000

def update_states(hidden_states,current_states,g=0.641,tau_S = 100):
    return -1*(current_states/tau_S) + (np.ones(2) - current_states) * g * input_output_relation(hidden_states)/1000

In [8]:
#Parameters
mu_0 = 20 #input strength; motion strength
c = 0 #input coherence

#Parameters to the input-output function
a = 270 #[(VnC)^{-1}]
b = 108 #[Hz]
d = 0.154 #[s]

#Kinetic parameters
g = 0.641 #dimensionless, called gamma in the paper
tau_S = 100 #[ms]
tau_AMPA = 2 #[ms]

#Synaptic coupling (population autapses {11,22} and between populations {21,12})
#JN11 = 0.2609
#JN22 = 0.2609
self_coupling = 0.2609
#JN21 = 0.0497
#JN12 = 0.0497
outer_coupling = 0.0497
couplings = [self_coupling,-outer_coupling]
#a 2x2 matrix whose main diagonal is comprised of self_coupling and whose secondary diagonal is -outer_coupling
couplings_matrix = [couplings,couplings[::-1]]

#Inputs from other sources (inhibitory and neutral populations: JA_ext; external: I0)
JA_ext = 5.2*np.exp(-4)
I0 = 0.3255

#Simulation noise
s_noise=0

#Visual motion stimulus to to the i-th {1,2} population 
I1 = JA_ext * mu_0 * (1 + c/100)
I2 = JA_ext * mu_0 * (1 - c/100)
inputs = np.array([I1,I2])

#Initial activity (firing rate) of each population
S1 = 1.1
S2 = 1.1
population_states = np.array([S1,S2])
current_noise = np.zeros(2)

#Simulation
sim_steps = 20_000
dt = 0.001
timestamps = []
time = 0

In [10]:
for i in range(sim_steps):
    time+=dt
    timestamps.append(time)
    current_noise = noisy_channel(current_noise)
    hidden_states = build_hidden_states(couplings=couplings_matrix,current_states=population_states,inputs=inputs,
                                    ionic_noises=current_noise,external_sources = I0)
    population_states = update_states(hidden_states = hidden_states,current_states=population_states)